# Lighthouses

Project plan:
1. Data Acquisition
a) get 4 different datasets
b) join them together
2. Data Cleaning

In [319]:
# import libraries
import time
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

In [320]:
# urls to scrape
urls_to_scrape = [
    
    'https://en.wikipedia.org/wiki/List_of_lighthouses_in_England',
    'https://en.wikipedia.org/wiki/List_of_lighthouses_in_Wales',
    'https://en.wikipedia.org/wiki/List_of_lighthouses_in_Scotland',
    'https://en.wikipedia.org/wiki/List_of_lighthouses_in_Ireland'
]

In [321]:
# dict for dataframes storage
d = {}
for i in range(3):
    d[i] = pd.DataFrame()

In [322]:
def get_data(urls_to_scrape):
    """
    iterates across urls provided, prints response for each iteration, 
    scrapes the wikitable, stores results in the dict
    """
    i = 0
    for url in urls_to_scrape:

        time.sleep(4)

        # get the response in the form of html
        wikiurl= url
        table_class="wikitable sortable jquery-tablesorter"
        response=requests.get(wikiurl)
        print('status is', response.status_code, 'for dataframe', i)

        soup = BeautifulSoup(response.text, 'html.parser')
        indiatable=soup.find('table',{'class':"wikitable"})

        df=pd.read_html(str(indiatable))
        d[i] =pd.DataFrame(df[0])
        i += 1
    return d
 
d = get_data(urls_to_scrape)

status is 200 for dataframe 0
status is 200 for dataframe 1
status is 200 for dataframe 2
status is 200 for dataframe 3


In [323]:
def get_columns(d):
    """
    get columns for each dataset
    """
    for i in range(4):
        print(d[i].columns)
        
get_columns(d)

Index(['Lighthouse', 'Image', 'Location & coordinates', 'County', 'Year built',
       'Tower height', 'Focal height', 'Range', 'Operator', 'NGA number'],
      dtype='object')
Index(['Lighthouse', 'Image', 'Location', 'Year built', 'Tower height',
       'Focal height', 'Range'],
      dtype='object')
Index(['Name', 'Image', 'Area coordinates', 'Island / mainland', 'Year built',
       'Designed / built by', 'Operated by / closed', 'Tower height',
       'Focal height', 'Range'],
      dtype='object')
Index(['Name', 'Image', 'Location Coordinates', 'County', 'Water body',
       'Year built', 'Tower height', 'Focal height', 'Range'],
      dtype='object')


the common columns are:
1. Lighthouse name
2. Location/coordinates
3. County/location
4. Year built
5. Tower height
6. Local height
7. Range

In [324]:
# get a specific column from each dataframe
columns_to_keep = ['Name', 'Lighthouse', 'Location & coordinates', 'Year built',
                   'Tower height', 'Focal height', 'Range',  'Location', 'Area coordinates', 
                  'Tower height','Focal height', 'Location Coordinates']

In [325]:
def rename_columns(d):
    """
    iterate across dfs in dict, 
    delete column in it is not in columns_to_keep
    """
    for i in range(4):
        for col in d[i].columns:
            try:
                if col not in columns_to_keep:
                    d[i].drop(col, axis = 1, inplace = True)
            except:
                pass
    return d

d = rename_columns(d)

In [326]:
def check_len_name(d):
    """
    check columns length and name
    """
    for i in range(4):
        print(d[i].columns)
        print(len(d[i].columns))
        
check_len_name(d)        

Index(['Lighthouse', 'Location & coordinates', 'Year built', 'Tower height',
       'Focal height', 'Range'],
      dtype='object')
6
Index(['Lighthouse', 'Location', 'Year built', 'Tower height', 'Focal height',
       'Range'],
      dtype='object')
6
Index(['Name', 'Area coordinates', 'Year built', 'Tower height',
       'Focal height', 'Range'],
      dtype='object')
6
Index(['Name', 'Location Coordinates', 'Year built', 'Tower height',
       'Focal height', 'Range'],
      dtype='object')
6


Before joining dataframes we need to assure that they have the same columns

In [327]:
def assign_unified_columns(d):
    """
    assure that each df has the same columns names
    """
    for i in range(4):
        d[i].columns = ['Name', 'Area_Coordinates', 'Year', 'Tower_height', 'Focal_height', 'Range']
    return d

d = assign_unified_columns(d)

In [328]:
check_len_name(d)

Index(['Name', 'Area_Coordinates', 'Year', 'Tower_height', 'Focal_height',
       'Range'],
      dtype='object')
6
Index(['Name', 'Area_Coordinates', 'Year', 'Tower_height', 'Focal_height',
       'Range'],
      dtype='object')
6
Index(['Name', 'Area_Coordinates', 'Year', 'Tower_height', 'Focal_height',
       'Range'],
      dtype='object')
6
Index(['Name', 'Area_Coordinates', 'Year', 'Tower_height', 'Focal_height',
       'Range'],
      dtype='object')
6


In [329]:
def join_dfs(d):
    """
    append dfs to list (df_to_append),
    concat dfs in that list
    """
    df_to_append = []
    for i in range(4):

        df_to_append.append(d[i])

    df_final = pd.concat(df_to_append)
    return df_final

df_final = join_dfs(d)

In [330]:
df_final.head()

,Name,Area_Coordinates,Year,Tower_height,Focal_height,Range
0,Anvil Point Lighthouse,Anvil Point50°35′31″N 1°57′36″W,1881,12 m (39 ft),45 m (148 ft),9 nmi (17 km; 10 mi)
1,Bamburgh Lighthouse,Bamburgh55°37′0″N 1°43′27″W,1910[10],9 m (30 ft)[10],12 m (39 ft)[11],"14 nmi (26 km; 16 mi)[11] (white), 11 nmi (20 ..."
2,Beachy Head Lighthouse,Beachy Head50°44′2″N 0°14′29″E[12],1902,43 m (141 ft),31 m (102 ft)[13],8 nmi (15 km; 9.2 mi)[13]
3,Berkeley Pill Front light,Severn Estuary51°41′59″N 2°29′24″W,1937,8 m (26 ft),5 m (16 ft),NaN
4,Berkeley Pill Rear light,Severn Estuary51°41′54″N 2°29′25″W,1937,12 m (39 ft),11 m (36 ft),NaN


# Data Cleaning

Plan for data cleaning:
1. Delete the word lighthouse from the 1st column
2. Create two columns with names Coordinates and Location from the 2nd column
3. Column Year must habe only year values, if there are two years (/) get the 2nd one
4. From the columns Tower_height and Focal_height get only values that correspond to meters not foots, convert to int
5. From the column Range get only values with km, convert to int
6. Clean the created coordinates column from anything except coordinates

1. Delete the word lighthouse from the 1st column

In [331]:
df_final.head()

,Name,Area_Coordinates,Year,Tower_height,Focal_height,Range
0,Anvil Point Lighthouse,Anvil Point50°35′31″N 1°57′36″W,1881,12 m (39 ft),45 m (148 ft),9 nmi (17 km; 10 mi)
1,Bamburgh Lighthouse,Bamburgh55°37′0″N 1°43′27″W,1910[10],9 m (30 ft)[10],12 m (39 ft)[11],"14 nmi (26 km; 16 mi)[11] (white), 11 nmi (20 ..."
2,Beachy Head Lighthouse,Beachy Head50°44′2″N 0°14′29″E[12],1902,43 m (141 ft),31 m (102 ft)[13],8 nmi (15 km; 9.2 mi)[13]
3,Berkeley Pill Front light,Severn Estuary51°41′59″N 2°29′24″W,1937,8 m (26 ft),5 m (16 ft),NaN
4,Berkeley Pill Rear light,Severn Estuary51°41′54″N 2°29′25″W,1937,12 m (39 ft),11 m (36 ft),NaN


In [332]:
def replace_lighthouse(df_final):
    """
    clean columns from Lighthouses
    """
    df_final['Name'] = df_final['Name'].str.replace('Lighthouse', '')
    return df_final

df_final = replace_lighthouse(df_final)

2. Create two columns with names Coordinates and Location from the 2nd column

In [333]:
def split_columns(df_final):
    """
    creates two columns: one with location, another one with coordinates
    drop unnecessary column thereafter
    """
    df_final['Location'] = df_final['Area_Coordinates'].str.extract('([a-zA-Z]+\s?[a-zA-Z]+)', expand=True)
    df_final['Coordinates'] = df_final['Area_Coordinates'].str.replace(r'[a-zA-Z]+\s?[a-zA-Z]+', '')
    df_final.drop('Area_Coordinates', axis = 1, inplace = True)
    return df_final

df_final = split_columns(df_final)

3. The column Year must have only year values, if there are two years (/) get the 2nd one

In [334]:
def get_year(x):
    """
    one there is the 2nd year value, get iy
    """
    try:
        x = x.split('/')[1]
    except:
        pass
    return x


df_final['Year'] = df_final['Year'].apply(lambda x: get_year(x))


def get_4_digit_year(df_final):
    """
    check if there are 4 digits in a year value
    """
    df_final['Year'] = df_final['Year'].str.findall('([0-9]{4})')
    return df_final


def list_to_str(x):
    """
    conert extracted list values to a string
    """
    try:
        return ''.join(x)
    except:
        pass
    return x

df_final['Year'] = df_final['Year'].apply(lambda x: list_to_str(x))



def check_numbers(x):
    """
    check numerical values
    """
    if type(x) == int or type(x) == float or len(x) == 4:
        return x
    else:
        return ''

df_final['Year'] = df_final['Year'].apply(lambda x: check_numbers(x))


def convert_year(df_final):
    """
    convert values to numeric, fill None with 0, convert to int
    """
    df_final['Year'] = pd.to_numeric(df_final['Year'], errors = 'coerce')
    df_final['Year'].fillna(0, inplace = True)
    df_final['Year'] = df_final['Year'].apply(lambda x: int(x))
    return df_final

df_final = convert_year(df_final)

4. From the columns Tower_height and Focal_height get only values that correspond to meters not foots, convert to int

In [335]:
def get_tower_height(df_final):
    """extract the tower height numbers, fillna with 0, convert to int type
    """
    df_final['Tower_height'] = df_final['Tower_height'].str.extract(r'(\d+)')
    df_final['Tower_height'].fillna(0, inplace = True)
    df_final['Tower_height'] = df_final['Tower_height'].astype(int)
    return df_final

df_final = get_tower_height(df_final)

In [336]:
def get_tower_height(df_final):
    """extract the tower height numbers, fillna with 0, convert to int type
    """
    df_final['Focal_height'] = df_final['Focal_height'].str.extract(r'(\d+)')
    df_final['Focal_height'].fillna(0, inplace = True)
    df_final['Focal_height'] = df_final['Focal_height'].astype(int)
    return df_final

df_final = get_tower_height(df_final)

5. From the column Range get only values with km, convert to int

In [337]:
def get_Range_values(df_final):
    """extract the tower height numbers, replace brakets with '', fillna with 0, convert to int type
    """
    df_final['Range'] = df_final['Range'].str.extract(r'(\(\d+)')
    df_final['Range'] = df_final['Range'].str.replace('(','')
    df_final['Range'].fillna(0, inplace = True)
    df_final['Range'] = df_final['Range'].astype(int)
    return df_final

df_final = get_Range_values(df_final)

6. Clean the created coordinates column from anything except coordinates

In [355]:
def coord(df_final):
    """
    removes [digits] from a string
    """
    df_final['Coordinates'] = df_final['Coordinates'].str.replace('(\[\d+])','')
    df_final['Coordinates'] = df_final['Coordinates'].str.replace('--','')
    df_final['Coordinates'] = df_final['Coordinates'].str.replace('(^[a-zA-Z]+)','')
    return df_final

df_final = coord(df_final)

# Cleaned DataFrame

In [356]:
df_final.head(10)

,Name,Year,Tower_height,Focal_height,Range,Location,Coordinates
0,Anvil Point,1881,12,45,17,Anvil Point,50°35′31″N 1°57′36″W
1,Bamburgh,0,9,12,26,Bamburgh,55°37′0″N 1°43′27″W
2,Beachy Head,1902,43,31,15,Beachy Head,50°44′2″N 0°14′29″E
3,Berkeley Pill Front light,1937,8,5,0,Severn Estuary,51°41′59″N 2°29′24″W
4,Berkeley Pill Rear light,1937,12,11,0,Severn Estuary,51°41′54″N 2°29′25″W
5,Berry Head,1906,5,58,35,Berry Head,50°23′58″N 3°29′0″W
6,Berwick,1826,13,15,11,Berwick,55°45′53″N 1°59′7″W
7,Bishop Rock,0,49,44,37,Bishop Rock,49°52′18″N 6°26′42″W
8,Bull Point,1879,11,54,37,Mortehoe,51°11′57″N 4°12′4″W
9,Burnham-on-Sea Low,0,9,7,22,Burnham,51°14′54″N 3°0′21″W
